In [11]:
# import packages
import pandas as pd
import numpy as np
from bt_classes import my_backtest
# orig_df = pd.read_csv('../res/test_data.csv',index_col=0,parse_dates=True)
# my_backtest(orig_df[:500])
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20,10
# df = pd.read_csv('../res/input0130.csv')

orig_df = pd.read_csv('../xau_1h_10y.csv')
orig_df['datetime'] = pd.to_datetime(orig_df['datetime'])
orig_df = orig_df.set_index('datetime')

df = orig_df.copy()
df['log_r'] = np.log(df['close']) - np.log(df['open'])
df['label'] = np.sign(df['log_r'].shift(-1))
df['label'][df['label']==-1] = 0
df['label'].fillna(0,inplace=True)

KeyError: 'datetime'

In [3]:
#importing required libraries
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
# follow the literature
# we don't use min-max scaling here, use partial mean-std scaling instead
from sklearn.preprocessing import StandardScaler
from itertools import chain

# and we define our model here
def lstm_model(para_a=42, para_b=17):
    model = Sequential()
    model.add(LSTM(units=para_a, dropout=0.1, return_sequences=True, input_shape=(240,1),activation='tanh'))# (25,15)-57, (42,17)-58
    model.add(LSTM(units=para_b, dropout=0.08, activation='tanh'))
    # model.add(Dropout(0.08))# 加了之后同原先效果差不多，（应该一定程度上）可以防止过拟合
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Using TensorFlow backend.
D:\Anaconda3\envs\inno\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\envs\inno\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\envs\inno\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\envs\inno\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Fu

## Method 1, train for only one time (many many days, maybe 2000?) and test several times(let's try 250 days per test). 

In [4]:
train_begin = 240
train_end = train_begin + 10000
scaler = StandardScaler()
train_set = df[['log_r','label']][train_begin-240:train_end].reset_index()
x_train, y_train = [], []
x_train_set = list(chain.from_iterable(scaler.fit_transform(train_set['log_r'].values.reshape(-1,1))))
for i in range(240,len(x_train_set)):
    x_train.append(x_train_set[i-240:i])
    y_train.append(train_set['label'][i])
x_train, y_train = np.array(x_train), np.array(y_train)
y_train = to_categorical(y_train,2)
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1)) 

model = lstm_model()
model.fit(x_train,y_train,epochs=30, batch_size=100, callbacks=[EarlyStopping(monitor='loss',patience=10)])


Epoch 1/30
 1400/10000 [===>..........................] - ETA: 54s - loss: 0.6944 - accuracy: 0.4757

KeyboardInterrupt: 

In [17]:
from bt_classes import *
def my_backtest(mydata, log=False, drawpic=False, iplot=False):
    plt.rcParams['figure.figsize'] = 12,8
    cerebro = bt.Cerebro()
    data = MyPandasData(dataname=mydata)
    cerebro.adddata(data)
    if log:
        cerebro.addstrategy(GoldStrategy)
    else:
        cerebro.addstrategy(GoldStrategy_nolog)
    cerebro.addsizer(OptInvest)
    init_value= 100000.0
    cerebro.broker.setcash(init_value)
    cerebro.broker.setcommission(commission=0,automargin=0.01,mult=100)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='dd')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='ar')
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    thestrats = cerebro.run()
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % final_value)
    if drawpic:
        cerebro.plot(iplot=iplot,volume=False)
    sharpe = thestrats[0].analyzers.sharpe.get_analysis()['sharperatio']
    dd = thestrats[0].analyzers.dd.get_analysis()['max']['drawdown']
    ar = thestrats[0].analyzers.returns.get_analysis()['rnorm100']
    print(thestrats[0].analyzers.sharpe.get_analysis())
    print('Sharpe: {:.2f}'.format(sharpe))
    print('Max drawdown: {:.2f}%'.format(dd))
    print('Annual rate: {:.2f}%'.format(ar))
    return sharpe,dd,ar

In [9]:
from keras.models import load_model
model = load_model('more_lstm.h5')
train_df = orig_df[train_begin:train_end]
train_df['label'] = model.predict_classes(x_train)
my_backtest(train_df,drawpic=True,iplot=True,log=True)

D:\Anaconda3\envs\inno\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Starting Portfolio Value: 100000.00
2019-08-23, Close, 1526.24, label 0
2019-08-23, SHORT CREATE, 1526.24
2019-08-23, SELL EXECUTED, 1526.50
2019-08-23, Close, 1526.37, label 0
2019-08-23, Close, 1526.20, label 0
2019-08-23, Close, 1526.35, label 0
2019-08-23, Close, 1527.36, label 0
2019-08-24, Close, 1527.34, label 0
2019-08-24, Close, 1526.45, label 0
2019-08-24, Close, 1525.04, label 1
2019-08-24, BUY CREATE, 1525.04
2019-08-24, BUY EXECUTED, 1524.97
2019-08-24, Close, 1523.52, label 1
2019-08-24, LONG CREATE, 1523.52
2019-08-24, BUY EXECUTED, 1523.52
2019-08-24, Close, 1523.95, label 1
2019-08-24, Close, 1523.28, label 1
2019-08-24, Close, 1524.47, label 1
2019-08-24, Close, 1523.78, label 1
2019-08-24, Close, 1524.40, label 1
2019-08-24, Close, 1525.44, label 1
2019-08-24, Close, 1526.26, label 0
2019-08-24, SELL CREATE, 1526.26
2019-08-24, SELL EXECUTED, 1526.40
2019-08-24, Close, 1526.54, label 0
2019-08-24, SHORT CREATE, 1526.54
2019-08-24, SELL EXECUTED, 1526.61
2019-08-24, C

TypeError: unsupported format string passed to NoneType.__format__

In [18]:
my_backtest(df[:20000],drawpic=True,iplot=True,log=False)

Starting Portfolio Value: 100000.00
Final Portfolio Value: 2156969.00
OrderedDict([('sharperatio', None)])


TypeError: unsupported format string passed to NoneType.__format__

In [ ]:
train_df['label'].describe()

In [ ]:
model.summary()

In [ ]:
model.save('more_lstm.h5')

In [ ]:
from tqdm import tqdm
from sa import *
from utils import *

# Please select the last activation layer.
layer_names = ['lstm_6']

default_upper_bound = 2000
default_n_bucket = 1000
default_n_classes = 2
class Args(): #创建一个类
    def __init__(self): #定义初始化信息。
        self.is_classification = True
        self.save_path = './tmp/'
        self.d = 'lstm_r'
        self.num_classes = 2
        self.lsa = True
        self.dsa = True
        self.target = 'none'
        self.batch_size = 128
        self.var_threshold = 1e-5
        self.upper_bound = 2000
        self.n_bucket = 1000
        self.is_classification = True
args = Args()

In [ ]:
starter = range(10240,len(df)-250,250)
all_results = []

for test_begin in starter:
    test_end = test_begin + 250

    x_test, y_test = [], []
    test_set = df[['log_r','label']][test_begin-240:test_end].reset_index()
    x_test_set = list(chain.from_iterable(scaler.transform(test_set['log_r'].values.reshape(-1,1))))
    for i in range(240,len(x_test_set)):
        x_test.append(x_test_set[i-240:i])
        y_test.append(test_set['label'][i])
    x_test, y_test = np.array(x_test), np.array(y_test)
    x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1)) 
    y_test = to_categorical(y_test,2)

    test_dsa = fetch_dsa(model, x_train, x_test, 'dsa from{}'.format(test_begin), layer_names, args)
    dsa_cov = get_sc(np.amin(test_dsa), 5, 20, test_dsa)
    test_lsa = fetch_lsa(model, x_train, x_test, 'lsa from{}'.format(test_begin), layer_names, args)
    lsa_cov = get_sc(np.amin(test_lsa), 1500, 20, test_lsa)
    eva = model.evaluate(x_test, y_test)
    print('Loss: {:.2f}, Accuracy: {:.2f}'.format(eva[0], eva[1]))
    print('Dsa Mean: {:.2f}, Var: {:.2f}, Max: {:.2f}, Min: {:.2f}'.format(np.mean(test_dsa),np.std(test_dsa),np.max(test_dsa),np.min(test_dsa)))
    print('Lsa Mean: {:.2f}, Var: {:.2f}, Max: {:.2f}, Min: {:.2f}'.format(np.mean(test_lsa),np.std(test_lsa),np.max(test_lsa),np.min(test_lsa)))
    test_df = orig_df[test_begin:test_end]
    test_df['label'] = model.predict_classes(x_test)
    sharpe,dd,ar = my_backtest(test_df)
    this_result = [test_begin,eva[0], eva[1],dsa_cov,np.mean(test_dsa),np.std(test_dsa),np.max(test_dsa),np.min(test_dsa),lsa_cov,np.mean(test_lsa),np.std(test_lsa),np.max(test_lsa),np.min(test_lsa),sharpe,dd,ar]
    all_results.append(this_result)

In [ ]:
cols = ['test_begin','loss', 'accu','dsa_cov','dsa_mean','dsa_std','dsa_max','dsa_min','lsa_cov','lsa_mean','lsa_std','lsa_max','lsa_min','sharpe','drawdown','annual_return']
result_df = pd.DataFrame(all_results,columns=cols)

In [ ]:
result_df.to_csv('rs.csv')